In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv(
    "D:\\IPO_analysis\\data\\Initial Public Offering.csv",
    parse_dates=["Date"]
)


C:\Users\Admin\AppData\Local\Temp\ipykernel_6192\426425445.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


In [8]:
df

,Date,IPO_Name,Issue_Size(crores),QIB,HNI,RII,Total,Offer Price,List Price,Listing Gain,...,CMP(NSE),Current Gains,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,2025-08-06,M & B Engineering Ltd,650,36.72,38.24,32.55,36.20,385,386,0.26,...,426.15,10.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-08-06,Sri Lotus Developers & Realty Ltd,792,163.90,57.71,20.28,69.14,150,179.1,19.40,...,199.72,34.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-08-06,National Securities Depository Ltd (NSDL),"4,011.60",103.97,34.98,7.73,41.01,800,880,10.00,...,61.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-08-05,Aditya Infotech Ltd,"1,300",133.21,72.00,50.87,100.69,675,"1,018.00",50.81,...,"1,062.70",57.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-08-05,Laxmi India Finance Ltd,254.26,1.30,1.84,2.22,1.87,158,136,-13.92,...,150,-5.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,2010-02-03,Infinite Computer Solutions Limited,189.8,48.44,106.02,11.08,43.22,165,178.35,8.09,...,473.75,187.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557,2010-01-06,D B Corp Limited,384.22,68.52,26.17,3.42,39.54,212,250,17.92,...,273,29.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
558,2010-01-05,Godrej Properties Limited,468.85,7.45,0.41,0.38,4.00,490,510,4.08,...,"1,952.10",299.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
559,2010-01-04,JSW Energy Limited,2700,2.88,0.15,0.40,1.68,100,102,2.00,...,536.4,436.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Step 1: Read normally
df = pd.read_csv("D:\\IPO_analysis\\data\\Initial Public Offering.csv")

# Step 2: Convert Date safely
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                561 non-null    datetime64[ns]
 1   IPO_Name            561 non-null    object        
 2   Issue_Size(crores)  561 non-null    object        
 3   QIB                 559 non-null    float64       
 4   HNI                 559 non-null    float64       
 5   RII                 559 non-null    float64       
 6   Total               559 non-null    float64       
 7   Offer Price         561 non-null    object        
 8   List Price          561 non-null    object        
 9   Listing Gain        561 non-null    float64       
 10  CMP(BSE)            559 non-null    object        
 11  CMP(NSE)            551 non-null    object        
 12  Current Gains       558 non-null    object        
 13  Unnamed: 13         0 non-null      float64       

C:\Users\Admin\AppData\Local\Temp\ipykernel_6192\2521350748.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")


In [11]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


In [12]:
# ---- 2. Handle missing values ----
# Subscription data: if missing, assume 0 (not subscribed)
for col in ["QIB", "HNI", "RII", "Total"]:
    df[col] = df[col].fillna(0)

In [13]:
# CMP columns: impute missing with NSE if BSE missing and vice versa
df["CMP(BSE)"] = df["CMP(BSE)"].fillna(df["CMP(NSE)"])
df["CMP(NSE)"] = df["CMP(NSE)"].fillna(df["CMP(BSE)"])

In [15]:
# Convert price columns to numeric (force errors to NaN)
cols = ["CMP(BSE)", "CMP(NSE)", "Offer Price"]
for c in cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Recompute Current Gains if missing
df["Current Gains"] = df["Current Gains"].fillna(
    ((df["CMP(BSE)"].fillna(df["CMP(NSE)"])) - df["Offer Price"]) / df["Offer Price"] * 100
)


In [16]:

# ---- 3. Add temporal features ----
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Quarter"] = df["Date"].dt.quarter


In [17]:
# ---- 4. Fix data types ----
numeric_cols = [
    "Issue_Size(crores)", "QIB", "HNI", "RII", "Total",
    "Offer Price", "List Price", "Listing Gain",
    "CMP(BSE)", "CMP(NSE)", "Current Gains"
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")


In [18]:
# ---- 5. Remove duplicates ----
df = df.drop_duplicates(subset=["IPO_Name", "Date"])


In [19]:
# ---- 6. Handle extreme values / sanity checks ----
# Cap unreasonable listing gains/losses (e.g., >300% or <-90%) as outliers
df["Listing Gain"] = df["Listing Gain"].clip(-90, 300)
df["Current Gains"] = df["Current Gains"].clip(-95, 1000)


In [21]:
# ---- 7. Save cleaned data ----
df.to_csv(r"D:\IPO_analysis\data\ipo_clean.csv", index=False)


In [22]:
print("✅ Data cleaned and saved to data/ipo_clean.csv")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

✅ Data cleaned and saved to data/ipo_clean.csv
Shape: (561, 16)
Columns: ['Date', 'IPO_Name', 'Issue_Size(crores)', 'QIB', 'HNI', 'RII', 'Total', 'Offer Price', 'List Price', 'Listing Gain', 'CMP(BSE)', 'CMP(NSE)', 'Current Gains', 'Year', 'Month', 'Quarter']
